In [1]:
import json
from typing import Any, Dict, List
from enum import Enum

from pydantic import BaseModel, create_model

from bbox import Bbox

In [5]:
def create_dynamic_enum(name: str, values: List[Any]) -> Enum:
    return Enum(name, {str(v): v for v in values})

def get_dtype(dtype_name: str, item_specifications: Dict[str, Any]) -> type:
    if dtype_name == "str":
        dtype = str
    elif dtype_name == "int":
        dtype = int
    elif dtype_name == "float":
        dtype = float
    elif dtype_name == "bool":
        dtype = bool
    elif dtype_name == "bbox":
        dtype = Bbox
    elif dtype_name in item_specifications:
        dtype = make(
            specification=item_specifications[dtype_name],
            item_specifications=item_specifications,
            name=dtype_name
        )
    else:
        raise ValueError("dtype {} not supported".format(dtype_name))
    return dtype

def make(
    specification: Dict[str, Any],
    item_specifications: Dict[str, Any],
    name="model"
) -> BaseModel:
    """
    specification must be defined like
    {
        "{KEY_NAME}": {
                "dim": 0, ## 0: str, 1: List[{dtype}]
                "dtype": str, # str, int, float, bool
                "allowed_values": ["a", "b"], ## for enum val
                "optional": False
        }
    }
    """
    spec_dict = dict()
    for k, v in specification.items():
        dtype = v.get("dtype", str)
        dim = v.get("dim", 0)
        allowed_values = v.get("allowed_values", None)
        default_val = ...

        if isinstance(dtype, str):
            dtype = get_dtype(
                dtype,
                item_specifications=item_specifications
            )

        if allowed_values:
            allowed_values = [dtype(v) for v in allowed_values]
            enum = create_dynamic_enum(f"{k}-enum", allowed_values)
            dtype = enum

        ## Dim
        if dim == 1:
            dtype = List[dtype]

        spec_dict[k] = (dtype, default_val)

    model = create_model(name, __config__={"extra": "forbid"}, **spec_dict)
    return model

In [6]:
with open("schemas/example.json", "r") as f:
    specification = json.load(f)

In [7]:
model = make(
    specification=specification["main"],
    item_specifications=specification["items"]
)

In [8]:
print(json.dumps(model.model_json_schema(), indent=2))

{
  "$defs": {
    "Bbox": {
      "additionalProperties": false,
      "properties": {
        "x1": {
          "title": "X1",
          "type": "integer"
        },
        "y1": {
          "title": "Y1",
          "type": "integer"
        },
        "x2": {
          "title": "X2",
          "type": "integer"
        },
        "y2": {
          "title": "Y2",
          "type": "integer"
        }
      },
      "required": [
        "x1",
        "y1",
        "x2",
        "y2"
      ],
      "title": "Bbox",
      "type": "object"
    },
    "child": {
      "additionalProperties": false,
      "properties": {
        "item_bbox": {
          "$ref": "#/$defs/Bbox"
        },
        "item_description": {
          "title": "Item Description",
          "type": "string"
        }
      },
      "required": [
        "item_bbox",
        "item_description"
      ],
      "title": "child",
      "type": "object"
    }
  },
  "additionalProperties": false,
  "properties": {
    "

In [9]:
sample = {
    "items": [
        {
            "item_bbox": {
                "x1": -1,
                "y1": -1,
                "x2": -1,
                "y2": -1,
            },
            "item_description": "item1",
        },
        {
            "item_bbox": {
                "x1": -1,
                "y1": -1,
                "x2": -1,
                "y2": -1,
            },
            "item_description": "item2",
        },
    ]
}

In [13]:
sample_instance = model.model_validate(sample)

In [ ]:
sample_instance

model(items=[child(item_bbox=Bbox(x1=-1, y1=-1, x2=-1, y2=-1), item_description='item1'), child(item_bbox=Bbox(x1=-1, y1=-1, x2=-1, y2=-1), item_description='item2')])

In [14]:
for item in sample_instance.items:
    print(item)

item_bbox=Bbox(x1=-1, y1=-1, x2=-1, y2=-1) item_description='item1'
item_bbox=Bbox(x1=-1, y1=-1, x2=-1, y2=-1) item_description='item2'


In [15]:
from utils.schema_model_maker import SchemaModelMaker

In [16]:
model = SchemaModelMaker.make(
    specification=specification["main"],
    item_specifications=specification["items"]
)

In [18]:
print(json.dumps(model.model_json_schema(), indent=2))

{
  "$defs": {
    "Bbox": {
      "additionalProperties": false,
      "properties": {
        "x1": {
          "title": "X1",
          "type": "integer"
        },
        "y1": {
          "title": "Y1",
          "type": "integer"
        },
        "x2": {
          "title": "X2",
          "type": "integer"
        },
        "y2": {
          "title": "Y2",
          "type": "integer"
        }
      },
      "required": [
        "x1",
        "y1",
        "x2",
        "y2"
      ],
      "title": "Bbox",
      "type": "object"
    },
    "child": {
      "additionalProperties": false,
      "properties": {
        "item_bbox": {
          "$ref": "#/$defs/Bbox"
        },
        "item_description": {
          "title": "Item Description",
          "type": "string"
        }
      },
      "required": [
        "item_bbox",
        "item_description"
      ],
      "title": "child",
      "type": "object"
    }
  },
  "additionalProperties": false,
  "properties": {
    "